In [1]:
!pwd

/kaggle/working


In [2]:
!tar -xzf ../input/spark311/spark-3.0.1-bin-hadoop3.2.tgz -C .
!pip install ../input/findspark/findspark-2.0.1-py2.py3-none-any.whl

Processing /kaggle/input/findspark/findspark-2.0.1-py2.py3-none-any.whl
findspark is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [3]:
import os
import sys

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64/"
os.environ["SPARK_HOME"] = "/kaggle/working/spark-3.0.1-bin-hadoop3.2"
spark_path = os.environ['SPARK_HOME']
sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.9-src.zip")

import findspark
findspark.init()

import pyspark
number_cores = 4
memory_gb = 2

conf = (pyspark.SparkConf().setMaster('local[{}]'.format(number_cores)).set('spark.driver.memory', '{}g'.format(memory_gb)))

sc = pyspark.SparkContext(conf=conf)


22/12/16 20:49:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [197]:
spark = pyspark.sql.SparkSession(sc)

In [198]:
train_data = spark.read.csv('../input/titanic/train.csv', header = True, inferSchema=True)
test_data = spark.read.csv('../input/titanic/test.csv', header = True, inferSchema=True)

In [199]:
train_data.count()

891

In [200]:
test_data.count()

418

In [201]:
train_data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [202]:
test_data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [203]:
# from pyspark.mllib.regression import LabeledPoint
# from pyspark.mllib.classification import SVMWithSGD

# data = [
#        LabeledPoint(0.0,[0.0]),
#        LabeledPoint(1.0,[1.0]),
#        LabeledPoint(1.0,[2.0]),
#        LabeledPoint(1.0,[3.0])
# ]
# svm = SVMWithSGD.train(sc.parallelize(data), iterations = 10)
# svm.predict([1.0])

In [204]:
# svm.predict(sc.parallelize([[1.0]])).collect()

In [205]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import SVMWithSGD
from pyspark.mllib.linalg import Matrix, Matrices, Vectors
from pyspark.mllib.linalg import DenseMatrix, SparseVector

sparse_data = [
    
    LabeledPoint(0.0, SparseVector(2, {0: -1.0})),
    LabeledPoint(1.0, SparseVector(2, {1: 1.0})),
    LabeledPoint(0.0, SparseVector(2, {0: 0.0})),
    LabeledPoint(1.0, SparseVector(2, {1: 2.0}))
]
svm = SVMWithSGD.train(sc.parallelize(sparse_data), iterations = 10)
svm.predict(SparseVector(2, {1:1.0}))

1

In [206]:
train_data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [207]:
train_data.take(5)

[Row(PassengerId=1, Survived=0, Pclass=3, Name='Braund, Mr. Owen Harris', Sex='male', Age=22.0, SibSp=1, Parch=0, Ticket='A/5 21171', Fare=7.25, Cabin=None, Embarked='S'),
 Row(PassengerId=2, Survived=1, Pclass=1, Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex='female', Age=38.0, SibSp=1, Parch=0, Ticket='PC 17599', Fare=71.2833, Cabin='C85', Embarked='C'),
 Row(PassengerId=3, Survived=1, Pclass=3, Name='Heikkinen, Miss. Laina', Sex='female', Age=26.0, SibSp=0, Parch=0, Ticket='STON/O2. 3101282', Fare=7.925, Cabin=None, Embarked='S'),
 Row(PassengerId=4, Survived=1, Pclass=1, Name='Futrelle, Mrs. Jacques Heath (Lily May Peel)', Sex='female', Age=35.0, SibSp=1, Parch=0, Ticket='113803', Fare=53.1, Cabin='C123', Embarked='S'),
 Row(PassengerId=5, Survived=0, Pclass=3, Name='Allen, Mr. William Henry', Sex='male', Age=35.0, SibSp=0, Parch=0, Ticket='373450', Fare=8.05, Cabin=None, Embarked='S')]

In [208]:
train_data.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [209]:
test_data.show()

+-----------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|        892|     3|    Kelly, Mr. James|  male|34.5|    0|    0|          330911| 7.8292| null|       Q|
|        893|     3|Wilkes, Mrs. Jame...|female|47.0|    1|    0|          363272|    7.0| null|       S|
|        894|     2|Myles, Mr. Thomas...|  male|62.0|    0|    0|          240276| 9.6875| null|       Q|
|        895|     3|    Wirz, Mr. Albert|  male|27.0|    0|    0|          315154| 8.6625| null|       S|
|        896|     3|Hirvonen, Mrs. Al...|female|22.0|    1|    1|         3101298|12.2875| null|       S|
|        897|     3|Svensson, Mr. Joh...|  male|14.0|    0|    0|            7538|  9.225| null|       S|
|        898|     3|Connolly, Miss. Kate|femal

In [210]:
train_data.collect()[0:3]

[Row(PassengerId=1, Survived=0, Pclass=3, Name='Braund, Mr. Owen Harris', Sex='male', Age=22.0, SibSp=1, Parch=0, Ticket='A/5 21171', Fare=7.25, Cabin=None, Embarked='S'),
 Row(PassengerId=2, Survived=1, Pclass=1, Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex='female', Age=38.0, SibSp=1, Parch=0, Ticket='PC 17599', Fare=71.2833, Cabin='C85', Embarked='C'),
 Row(PassengerId=3, Survived=1, Pclass=3, Name='Heikkinen, Miss. Laina', Sex='female', Age=26.0, SibSp=0, Parch=0, Ticket='STON/O2. 3101282', Fare=7.925, Cabin=None, Embarked='S')]

In [211]:
train_data.groupBy('Survived').count().show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



In [212]:
train_data.groupBy('Survived').pivot('Sex').count().show()

+--------+------+----+
|Survived|female|male|
+--------+------+----+
|       1|   233| 109|
|       0|    81| 468|
+--------+------+----+



In [213]:
unique_survived = train_data.select('Survived').distinct().collect()
print(unique_survived)

[Row(Survived=1), Row(Survived=0)]


In [214]:
unique_sex = train_data.select('Sex').distinct().collect()
print(unique_sex)

[Row(Sex='female'), Row(Sex='male')]


In [215]:
unique_test_sex = test_data.select('Sex').distinct().collect()
print(unique_test_sex)

[Row(Sex='female'), Row(Sex='male')]


In [216]:
unique_sibling_spouse = train_data.select('Sibsp').distinct().collect()
print(unique_sibling_spouse)

[Row(Sibsp=1), Row(Sibsp=3), Row(Sibsp=5), Row(Sibsp=4), Row(Sibsp=8), Row(Sibsp=2), Row(Sibsp=0)]


In [217]:
unique_embarkation = train_data.select('Embarked').distinct().collect()
print(unique_embarkation)

[Row(Embarked='Q'), Row(Embarked=None), Row(Embarked='C'), Row(Embarked='S')]


In [218]:
train_columns = train_data.columns
print(train_columns)

['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']


In [219]:
test_columns = test_data.columns
print(test_columns)

['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']


In [220]:
# train_data = train_data.drop('Name', 'Ticket', 'Cabin')
# test_data = test_data.drop('Name', 'Ticket', 'Cabin')

In [221]:
train_columns = train_data.columns
print(train_columns)

['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']


In [222]:
test_columns = test_data.columns
print(test_columns)

['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']


In [223]:
train_data.replace('None',None).na.fill(0).show()
test_data.replace('None',None).na.fill(0).show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male| 0.0|    0|    0|      

In [224]:
# svm_survival = SVMWithSGD.train(df_svm, iterations=100)

In [225]:
from pyspark.sql.functions import when
from pyspark.sql.types import *
#import org.apache.spark.sql.functions.col
#import org.apache.spark.sql.types.IntegerType
from pyspark.sql.functions import col
from pyspark.sql.types import ArrayType,IntegerType

#https://sparkbyexamples.com/spark/spark-case-when-otherwise-example/

#df = train_data.withColumn("Gender",col("Sex").cast(IntegerType()))
#train_data.withColumn("Sex",col("Sex").cast("int"))
#train_data.withColumn("Sex",col("Sex").cast("integer"))

# df = train_data.select(col("Sex").cast("int").as('Sex'))
# df.printSchema

#https://sparkbyexamples.com/spark/spark-case-when-otherwise-example/
df = train_data.withColumn("Gender", when(col("Sex") == "male",int(0))
      .when(col("Sex") == "female",int(1)))
df.printSchema
df.show(2)


df_test = test_data.withColumn("Gender", when(col("Sex") == "male",0)
      .when(col("Sex") == "female",1))

df_test.printSchema
df_test.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+-----+--------+------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|   Ticket|   Fare|Cabin|Embarked|Gender|
+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+-----+--------+------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|A/5 21171|   7.25| null|       S|     0|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0| PC 17599|71.2833|  C85|       C|     1|
+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+-----+--------+------+
only showing top 2 rows

+-----------+------+--------------------+------+----+-----+-----+-------+-------+-----+--------+------+
|PassengerId|Pclass|                Name|   Sex| Age|SibSp|Parch| Ticket|   Fare|Cabin|Embarked|Gender|
+-----------+------+--------------------+------+----+-----+-----+-------+----

In [226]:
dfcolumns = df.columns
dfcolumns
req_col = [dfcolumns.index('Gender'),dfcolumns.index('Age'),dfcolumns.index('Pclass'),dfcolumns.index('Fare'), dfcolumns.index('SibSp')]
print(req_col)




[12, 5, 2, 9, 6]


In [227]:
dftest_columns = df_test.columns
req_test_col = [dftest_columns.index('Pclass'), dftest_columns.index('Age'), dftest_columns.index('SibSp'), dftest_columns.index('Fare'), dftest_columns.index('Gender')]
print(req_test_col)

[1, 4, 5, 8, 11]


In [228]:
# def dataPrep(r):
#     key = 0
#     value = []
#     for c in columns:
#         if c == 'deposite':
#             key = cat_dictionary[c][r[columns.index(c)]]
#         else:
#             if c in cat_columns:
#                 value.append(cat_dictionary[c][r[columns.index(c)]])
#             else:
#                 value.append(r[columns.index(c)])
#     return LabeledPoint(key,value)

In [229]:
dfcolumns = df.columns
req_col = [dfcolumns.index('Gender'),dfcolumns.index('Age'),dfcolumns.index('Pclass'),dfcolumns.index('Fare'), dfcolumns.index('SibSp')]

data=[]
#https://sparkbyexamples.com/spark/spark-dataframe-collect/
dataArray = df.collect()

for line in dataArray:
    #data=[]
    data.append(LabeledPoint(line['Survived'], [line[c] for c in  req_col]))
    #LabeledPoint(key,value)
    
dftest_columns = df_test.columns
req_test_col = [dftest_columns.index('Gender'), dftest_columns.index('Age'),dftest_columns.index('Pclass'), dftest_columns.index('Fare'),dftest_columns.index('SibSp')]

predictions=[]
testDataArray = df_test.collect()

for line in testDataArray:
    predictions.append(svm_train.predict([line[c] for c in req_test_col]))    


In [230]:
svm_train = SVMWithSGD.train(sc.parallelize(data), iterations = 200)

In [231]:
dftest_columns = df_test.columns
dftest_columns

['PassengerId',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked',
 'Gender']

In [232]:
from pyspark.sql.functions import monotonically_increasing_id, row_number
from pyspark.sql import Window

a = df_test.select('PassengerID')
pred_col = ["Survived"]
b = spark.createDataFrame(pred_list, IntegerType())
#https://www.educba.com/pyspark-alias/
b = b.select(col("value").alias("Survived"))
#https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.monotonically_increasing_id.html
#a = a.withColumn("row_idx", row_number().over(orderBy(monotonically_increasing_id())))
#https://stackoverflow.com/questions/48164206/pyspark-adding-a-column-from-a-list-of-values/48174607#48174607
a = a.withColumn("row_idx", row_number().over(Window.orderBy(monotonically_increasing_id())))
b = b.withColumn("row_idx", row_number().over(Window.orderBy(monotonically_increasing_id())))

pred_df = a.join(b, a.row_idx == b.row_idx).\
             drop("row_idx")

In [233]:
#https://stackoverflow.com/questions/52411992/how-to-produce-a-kaggle-submission-csv-file-with-specific-entries
pred_df.toPandas().to_csv('submission.csv', index = False)

22/12/16 23:03:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/12/16 23:03:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [234]:
#https://stackoverflow.com/questions/43298291/python-jupyter-notebook-unable-to-open-csv-file-through-a-path
import os
os.chdir(r'../working')
from IPython.display import FileLink
FileLink(r'submission.csv')

/kaggle/working/submission.csv